# Setup

In [12]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget

# Load resources
rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Base MPC controller class

In [13]:
from abc import abstractmethod
import cvxpy as cp
from cvxpy import Expression, Constraint
import numpy as np
from control import dlqr
from mpt4py import Polyhedron
from scipy.signal import cont2discrete

class MPCControl_base:

	x_ids: np.ndarray
	u_ids: np.ndarray

	def __init__(
		self,
		A: np.ndarray,
		B: np.ndarray,
		xs: np.ndarray,
		us: np.ndarray,
		Ts: float,
		H: float,
	) -> None:
		
		# Save controller configuration
		self.N = int(H / Ts)
		self.NX = self.x_ids.shape[0]
		self.NU = self.u_ids.shape[0]
		self.Ts = Ts
		self.xs = xs
		self.us = us

		# Extract subset of discretized states and inputs
		subA = A[np.meshgrid(self.x_ids, self.x_ids)].T
		subB = B[np.meshgrid(self.x_ids, self.u_ids)].T
		self.A, self.B = self._discretize(subA, subB, Ts)

		# Create optimization variables and parameters
		self.x_var 	= cp.Variable((self.NX, self.N + 1), name='x')
		self.u_var 	= cp.Variable((self.NU, self.N), name='u')
		self.x0_par	= cp.Parameter(self.NX, name='x0')
		self.xs_par	= cp.Parameter(self.NX, name='xs')
		self.us_par	= cp.Parameter(self.NU, name='us')

		# Create optimization problem
		cost, constraints = self._get_cost_and_constraints()
		self.ocp = cp.Problem(cp.Minimize(cost), constraints)
		self.xs_par.value = xs[self.x_ids]
		self.us_par.value = us[self.u_ids]

	@abstractmethod
	def _get_cost_and_constraints(self) -> tuple[Expression, list[Constraint]]:
		pass

	@staticmethod
	def _max_invariant_set(O: Polyhedron, A_cl: np.ndarray, max_iter: int = 30) -> Polyhedron:
		for _ in range(max_iter):
			Oprev = O
			O = Polyhedron.from_Hrep(np.vstack([O.A, O.A @ A_cl]), np.vstack([O.b, O.b]).reshape(-1))
			O.minHrep(True)
			_ = O.Vrep
			if O == Oprev:
				return O
		raise RuntimeError('Did not converge to maximum invariant set')

	@staticmethod
	def _discretize(A: np.ndarray, B: np.ndarray, Ts: float):
		NX, NU = B.shape
		C = np.zeros((1, NX))
		D = np.zeros((1, NU))
		A_discrete, B_discrete, _, _, _ = cont2discrete(system=(A, B, C, D), dt=Ts)
		return A_discrete, B_discrete

	def get_u(
		self,
		x0: np.ndarray,
		x_target: np.ndarray = None,
        u_target: np.ndarray = None
	) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
		
		# Allocate outputs
		x_traj = np.zeros((self.NX, self.N + 1))
		u_traj = np.zeros((self.NU, self.N))

		# Compute steady state error
		x_traj[:, 0] = x0 - self.xs_par.value
		xk: np.ndarray = x_traj[:, 0]

		# Closed-loop simulation
		for k in range(self.N):
			self.x0_par.value = xk
			self.ocp.solve(solver=cp.PIQP)
			assert self.ocp.status == cp.OPTIMAL
			uk: np.ndarray = self.u_var.value[:, 0] - self.us_par.value
			xk = self.A @ xk + self.B @ uk
			x_traj[:, k + 1] = xk.flatten()
			u_traj[:, k] = uk.flatten()

		# Return predicted input and trajectories
		u0 = u_traj[:, 0]
		return u0, x_traj, u_traj

## X subsystem controller

In [14]:
class MPCControl_xvel(MPCControl_base):

	x_ids = np.array([1, 4, 6])
	u_ids = np.array([1])

	def _get_cost_and_constraints(self) -> tuple[Expression, list[Constraint]]:
		
		# Define stage cost
		Q = np.diag([1.0, 1.0, 1.0])
		R = np.diag([1.0])

		# Compute terminal controller
		K, Qf, _ = dlqr(self.A, self.B, Q, R)
		K = -K

		# Define trajectory cost
		cost = 0
		for i in range(self.N):
			cost += cp.quad_form(self.x_var[:, i] - self.xs_par, Q)
			cost += cp.quad_form(self.u_var[:, i] - self.us_par, R)
		cost += cp.quad_form(self.x_var[:, -1] - self.xs_par, Qf)
		
		# Define state constraints
		F = np.array([
			[0.0, +1.0, 0.0], 		# beta <= +10°
			[0.0, -1.0, 0.0] 		# beta >= -10°
		])
		f = np.array([
			np.pi / 18.0,			# beta <= +10°
			np.pi / 18.0			# beta >= -10°
		])
		X = Polyhedron.from_Hrep(F, f)
		
		# Define input constraints
		G = np.array([
			[+1.0],					# delta_2 <= +15°
			[-1.0]					# delta_2 >= -15°
		])
		g = np.array([
			np.pi / 12.0,			# delta_2 <= +15°
			np.pi / 12.0			# delta_2 >= -15°
		])
		U = Polyhedron.from_Hrep(G, g)

		# Compute max invariant set
		A_cl = self.A + self.B @ K
		O = X.intersect(Polyhedron.from_Hrep(U.A @ K, U.b))
		O = self._max_invariant_set(O, A_cl)

		# Define constraints
		constraints = [
			self.x_var[:, 0] 			== self.x0_par,
			self.x_var[:, 1:] 			== self.A @ self.x_var[:, :-1] + self.B @ self.u_var,
			X.A @ self.x_var[:, :-1] 	<= X.b.reshape(-1, 1),	# State lies in state constraints
			U.A @ self.u_var 			<= U.b.reshape(-1, 1),	# Input lies in input constraints
			O.A @ self.x_var[:, -1] 	<= O.b.reshape(-1, 1)	# Final state lies in terminal set
		]

		# Return cost and constraints
		return cost, constraints

## Y subsystem controller

In [15]:
class MPCControl_yvel(MPCControl_base):

	x_ids = np.array([0, 3, 7])
	u_ids = np.array([0])

	def _get_cost_and_constraints(self) -> tuple[Expression, list[Constraint]]:
		
		# Define stage cost
		Q = np.diag([1.0, 1.0, 1.0])
		R = np.diag([1.0])

		# Compute terminal controller
		K, Qf, _ = dlqr(self.A, self.B, Q, R)
		K = -K

		# Define trajectory cost
		cost = 0
		for i in range(self.N):
			cost += cp.quad_form(self.x_var[:, i] - self.xs_par, Q)
			cost += cp.quad_form(self.u_var[:, i] - self.us_par, R)
		cost += cp.quad_form(self.x_var[:, -1] - self.xs_par, Qf)
		
		# Define state constraints
		F = np.array([
			[0.0, +1.0, 0.0], 		# alpha <= +10°
			[0.0, -1.0, 0.0] 		# alpha >= -10°
		])
		f = np.array([
			np.pi / 18.0,			# alpha <= +10°
			np.pi / 18.0			# alpha >= -10°
		])
		X = Polyhedron.from_Hrep(F, f)
		
		# Define input constraints
		G = np.array([
			[+1.0],					# delta_1 <= +15°
			[-1.0]					# delta_1 >= -15°
		])
		g = np.array([
			np.pi / 12.0,			# delta_1 <= +15°
			np.pi / 12.0			# delta_1 >= -15°
		])
		U = Polyhedron.from_Hrep(G, g)

		# Compute max invariant set
		A_cl = self.A + self.B @ K
		O = X.intersect(Polyhedron.from_Hrep(U.A @ K, U.b))
		O = self._max_invariant_set(O, A_cl)

		# Define constraints
		constraints = [
			self.x_var[:, 0] 			== self.x0_par,
			self.x_var[:, 1:] 			== self.A @ self.x_var[:, :-1] + self.B @ self.u_var,
			X.A @ self.x_var[:, :-1] 	<= X.b.reshape(-1, 1),	# State lies in state constraints
			U.A @ self.u_var 			<= U.b.reshape(-1, 1),	# Input lies in input constraints
			O.A @ self.x_var[:, -1] 	<= O.b.reshape(-1, 1)	# Final state lies in terminal set
		]

		# Return cost and constraints
		return cost, constraints

## Z subsystem controller

In [ ]:
class MPCControl_zvel(MPCControl_base):

	x_ids = np.array([8])
	u_ids = np.array([2])

	def _get_cost_and_constraints(self) -> tuple[Expression, list[Constraint]]:
		
		# Define stage cost
		Q = np.diag([1.0])
		R = np.diag([1.0])

		# Define trajectory cost
		cost = 0
		for i in range(self.N):
			cost += cp.quad_form(self.x_var[:, i] - self.xs_par, Q)
			cost += cp.quad_form(self.u_var[:, i] - self.us_par, R)
		cost += cp.quad_form(self.x_var[:, -1] - self.xs_par, Q)

		# Define constraints
		constraints = [
			self.x_var[:, 0] 	== self.x0_par,
			self.x_var[:, 1:] 	== self.A @ self.x_var[:, :-1] + self.B @ self.u_var,
			self.u_var[:, :]	>= 40.0,	# P_avg >= 40% 
			self.u_var[:, :]	<= 80.0		# P_avg <= 80%
		]
	
		# Return cost and constraints
		return cost, constraints

## Roll subsystem controller

In [ ]:
class MPCControl_roll(MPCControl_base):

	x_ids = np.array([2, 5])
	u_ids = np.array([3])

	def _get_cost_and_constraints(self) -> tuple[Expression, list[Constraint]]:
		
		# Define stage cost
		Q = np.diag([1.0, 1.0])
		R = np.diag([1.0])

		# Define trajectory cost
		cost = 0
		for i in range(self.N):
			cost += cp.quad_form(self.x_var[:, i] - self.xs_par, Q)
			cost += cp.quad_form(self.u_var[:, i] - self.us_par, R)
		cost += cp.quad_form(self.x_var[:, -1] - self.xs_par, Q)

		# Define constraints
		constraints = [
			self.x_var[:, 0]	== self.x0_par,
			self.x_var[:, 1:]	== self.A @ self.x_var[:, :-1] + self.B @ self.u_var,
			self.u_var[:, :]	>= -20.0,	# P_diff >= -20%
			self.u_var[:, :]	<= +20.0	# P_diff <= +20%
		]
	
		# Return cost and constraints
		return cost, constraints

## Velocity controller

In [18]:
from src.rocket import Rocket

class MPCVelControl:
    
    mpc_x: MPCControl_xvel
    mpc_y: MPCControl_yvel
    mpc_z: MPCControl_zvel
    mpc_roll: MPCControl_roll

    def __init__(self) -> None:
        pass

    def new_controller(self, rocket: Rocket, Ts: float, H: float) -> 'MPCVelControl':
        self.xs, self.us = rocket.trim()
        A, B = rocket.linearize(self.xs, self.us)

        self.mpc_x = MPCControl_xvel(A, B, self.xs, self.us, Ts, H)
        self.mpc_y = MPCControl_yvel(A, B, self.xs, self.us, Ts, H)
        self.mpc_z = MPCControl_zvel(A, B, self.xs, self.us, Ts, H)
        self.mpc_roll = MPCControl_roll(A, B, self.xs, self.us, Ts, H)

        return self

    def load_controllers(
        self,
        mpc_x: MPCControl_xvel,
        mpc_y: MPCControl_yvel,
        mpc_z: MPCControl_zvel,
        mpc_roll: MPCControl_roll,
    ) -> None:
        self.mpc_x = mpc_x
        self.mpc_y = mpc_y
        self.mpc_z = mpc_z
        self.mpc_roll = mpc_roll

        return self

    def estimate_parameters(self, x_data: np.ndarray, u_data: np.ndarray) -> None:
        return

    def get_u(
        self,
        t0: float,
        x0: np.ndarray,
        x_target: np.ndarray = None,
        u_target: np.ndarray = None
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        u0 = np.zeros(4)
        t_traj = np.arange(self.mpc_x.N + 1) * self.mpc_x.Ts + t0
        x_traj = np.zeros((12, self.mpc_x.N + 1))
        u_traj = np.zeros((4, self.mpc_x.N))

        if x_target is None:
            x_target = self.xs

        if u_target is None:
            u_target = self.us

        u0[self.mpc_x.u_ids], x_traj[self.mpc_x.x_ids], u_traj[self.mpc_x.u_ids] = (
            self.mpc_x.get_u(
                x0[self.mpc_x.x_ids],
                x_target[self.mpc_x.x_ids],
                u_target[self.mpc_x.u_ids],
            )
        )
        u0[self.mpc_y.u_ids], x_traj[self.mpc_y.x_ids], u_traj[self.mpc_y.u_ids] = (
            self.mpc_y.get_u(
                x0[self.mpc_y.x_ids],
                x_target[self.mpc_y.x_ids],
                u_target[self.mpc_y.u_ids],
            )
        )
        u0[self.mpc_z.u_ids], x_traj[self.mpc_z.x_ids], u_traj[self.mpc_z.u_ids] = (
            self.mpc_z.get_u(
                x0[self.mpc_z.x_ids],
                x_target[self.mpc_z.x_ids],
                u_target[self.mpc_z.u_ids],
            )
        )
        (
            u0[self.mpc_roll.u_ids],
            x_traj[self.mpc_roll.x_ids],
            u_traj[self.mpc_roll.u_ids],
        ) = self.mpc_roll.get_u(
            x0[self.mpc_roll.x_ids],
            x_target[self.mpc_roll.x_ids],
            u_target[self.mpc_roll.u_ids],
        )

        return u0, x_traj, u_traj, t_traj


# Simulation

In [19]:
from src.vel_rocket_vis import RocketVis
from time import sleep

## Open loop trajectory

In [20]:
# Create rocket object
Ts = 1.0 / 20.0
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
H = 10.0
mpc = MPCVelControl().new_controller(rocket, Ts, H)

# Define initial state
t0 = 0.0
x0 = np.array([
    0.0, 0.0, 0.0,	# omega
    0.0, 0.0, np.pi * 40.0 / 180.0,	# r
    5.0, 5.0, 5.0,	# v
    0.0, 0.0, 0.0	# p
])

# Compute open loop prediction
_, x_ol, u_ol, t_ol, = mpc.get_u(t0, x0)

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [11]:
# Plot rocket path
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_ol[:-1], x_ol[:, :-1], u_ol)
sleep(0.2)

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=199, step=2), IntSlider(value=0…

## Closed loop trajectory

In [24]:
# Create rocket object
Ts = 1.0 / 20.0
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
H = 2.5
mpc = MPCVelControl().new_controller(rocket, Ts, H)

# Define initial state
t0 = 0.0
x0 = np.array([
    0.0, 0.0, 0.0,	# omega
    0.0, 0.0, np.pi * 40.0 / 180.0,	# r
    5.0, 5.0, 5.0,	# v
    0.0, 0.0, 0.0	# p
])
u0 = np.array([0.0, 0.0, 40.0, 0.0])

# Simulate closed loop control
sim_time = 5.0
x1 = rocket.simulate_step(x0, Ts, u0, method='nonlinear')
t_cl, x_cl, u_cl, _, _, _, _ = rocket.simulate_control(mpc, sim_time, H, x0, method='linear')

Simulating time 0.00: 

c:\Users\killi\Desktop\MPC-Course-EPFL\.env\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(



Simulating time 0.05: 
Simulating time 0.10: 
Simulating time 0.15: 
Simulating time 0.20: 
Simulating time 0.25: 
Simulating time 0.30: 
Simulating time 0.35: 
Simulating time 0.40: 
Simulating time 0.45: 
Simulating time 0.50: 
Simulating time 0.55: 
Simulating time 0.60: 
Simulating time 0.65: 
Simulating time 0.70: 
Simulating time 0.75: 
Simulating time 0.80: 
Simulating time 0.85: 
Simulating time 0.90: 
Simulating time 0.95: 
Simulating time 1.00: 
Simulating time 1.05: 
Simulating time 1.10: 
Simulating time 1.15: 
Simulating time 1.20: 
Simulating time 1.25: 
Simulating time 1.30: 
Simulating time 1.35: 
Simulating time 1.40: 
Simulating time 1.45: 
Simulating time 1.50: 
Simulating time 1.55: 
Simulating time 1.60: 
Simulating time 1.65: 
Simulating time 1.70: 
Simulating time 1.75: 
Simulating time 1.80: 
Simulating time 1.85: 
Simulating time 1.90: 
Simulating time 1.95: 
Simulating time 2.00: 
Simulating time 2.05: 
Simulating time 2.10: 
Simulating time 2.15: 
Simulating

In [ ]:
# Plot rocket path
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl)
sleep(0.2)

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=99, step=2), IntSlider(value=0,…